<p style='color: darkorange; font-size: 2.5em; font-weight: bold; text-align: left;'>
    Verarbeiten großer Datenbestände mit DuckDB 🦆</p>

### **Installation von DuckDB und Verbindung zu DuckDB (hier in-Memory-Verarbeitung)**

In [1]:
%%capture
%pip install duckdb

In [2]:
import duckdb

# Verbindung zur in-memory DuckDB-Datenbank (':memory:') herstellen; Voraussetzung: DuckDB muss installiert sein, ':memory:' muss als Vorgabe so gesetzt sein
conn = duckdb.connect(':memory:')

### **Inspektion der Daten via SQL**

In [3]:
# Inspektion der Daten und Test der Verbindung zur Datenquelle, die über die untere URL zu erreichen ist
result = conn.execute("""
DESCRIBE SELECT * FROM 'https://media.githubusercontent.com/media/swrobuts/Flights/main/airports.csv';
""").fetchall()

# Ausgabe des Ergebnisses
for row in result:
    print(row)

('IATA_CODE', 'VARCHAR', 'YES', None, None, None)
('AIRPORT', 'VARCHAR', 'YES', None, None, None)
('CITY', 'VARCHAR', 'YES', None, None, None)
('STATE', 'VARCHAR', 'YES', None, None, None)
('COUNTRY', 'VARCHAR', 'YES', None, None, None)
('LATITUDE', 'DOUBLE', 'YES', None, None, None)
('LONGITUDE', 'DOUBLE', 'YES', None, None, None)


### **Aufbau und Befüllen der Tabelle `airports`**

In [4]:
import pandas as pd

# Verbindung zur in-memory DuckDB-Datenbank herstellen; Voraussetzung: DuckDB muss installiert sein
conn = duckdb.connect(':memory:')

# Sequenz und Tabelle "airports" erstellen
conn.execute("""
CREATE SEQUENCE IF NOT EXISTS airport_id
INCREMENT BY 1 MINVALUE 0;
""")

conn.execute("""
CREATE TABLE IF NOT EXISTS airports (
    airport_id INTEGER PRIMARY KEY DEFAULT(nextval('airport_id')),
    iata_code VARCHAR(5),
    airport VARCHAR(255),
    city VARCHAR(255),
    state VARCHAR(255),
    country VARCHAR(255),
    latitude DOUBLE,
    longitude DOUBLE,
    CONSTRAINT airport_id UNIQUE (airport_id)
);
""")

# Befüllen der angelegten Tabelle bzw. Relation "airports" aus der csv-Datei
conn.execute("""
INSERT INTO airports("iata_code", "airport","city","state","country","latitude","longitude")
SELECT DISTINCT "IATA_CODE", "AIRPORT", "CITY", "STATE","COUNTRY","LATITUDE","LONGITUDE"
FROM 'https://media.githubusercontent.com/media/swrobuts/Flights/main/airports.csv';
""")

# Überführt die Daten aus der Relation "airports" in ein pandas DataFrame
airports_df = conn.execute("SELECT * FROM airports;").df()

# Einige Daten aus der Tabelle "airports" abfragen, um zu testen
airports_df.head()

,airport_id,iata_code,airport,city,state,country,latitude,longitude
0,86,AEX,Alexandria International Airport,Alexandria,LA,USA,31.32737,-92.54856
1,87,ALB,Albany International Airport,Albany,NY,USA,42.74812,-73.80298
2,88,ALO,Waterloo Regional Airport,Waterloo,IA,USA,42.55708,-92.40034
3,89,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619
4,90,ATW,Appleton International Airport,Appleton,WI,USA,44.25741,-88.51948


### **Aufbau und Befüllen der Tabelle `airlines`**

In [5]:
# Anlegen und Befüllen der Tabelle "airlines"
conn.execute("""
CREATE SEQUENCE IF NOT EXISTS "airline_id"
INCREMENT BY 1 MINVALUE 100;

CREATE TABLE IF NOT EXISTS airlines
(
    "airline_id" INTEGER PRIMARY KEY DEFAULT(nextval('airline_id')),
    "iata_code" VARCHAR(10),
    "airline" VARCHAR(255),
    CONSTRAINT "airline_id" UNIQUE ("airline_id")
    );

--- Befüllen der angelegten Tabelle "airlines"
INSERT INTO airlines("iata_code","airline")
SELECT DISTINCT "IATA_CODE", "AIRLINE"
FROM read_csv('https://media.githubusercontent.com/media/swrobuts/Flights/main/airlines.csv', 
              delim = ',',
              header = true,
              Columns = {
              'IATA_CODE': 'VARCHAR(5)',
              'AIRLINE': 'VARCHAR(255)'
              })
""")

# Überführt die Daten aus der Relation "airlines" in ein pandas DataFrame
airlines_df = conn.execute("SELECT * FROM airlines;").df()

# Einige Daten aus der Tabelle "airlines" abfragen, um zu testen
airlines_df.head()

,airline_id,iata_code,airline
0,100,OO,Skywest Airlines Inc.
1,112,US,US Airways Inc.
2,113,EV,Atlantic Southeast Airlines
3,106,NK,Spirit Air Lines
4,107,MQ,American Eagle Airlines Inc.


### **Anlegen und Befüllen der Tabelle `cancellation_reasons`**

In [6]:
## Anlegen der Tabelle "cancellation_reasons"
conn.execute("""
CREATE TABLE IF NOT EXISTS cancellation_reasons
(
    "cr_id" INTEGER PRIMARY KEY,
    "reason" VARCHAR(255),
    "shortcut" VARCHAR(5),
);

-- Manuelles Befüllen der Tabelle "cancellations_reasons", die IDs für den Primary Key sind willkürlich, sie müssen aber unterschiedlich sein
INSERT INTO cancellation_reasons
VALUES
(90, 'Airline/ Carrier', 'A'),
(80,'Weather', 'B'),
(70,'National Air System', 'C'),
(60,'Security', 'D'),
(0, 'Keine Stornierung', 'No');
""")

# Überführt die Daten aus der angelegten Relation in ein pandas DataFrame
cancellation_reasons_df = conn.execute("SELECT * FROM cancellation_reasons;").df()

# Testabfrage
cancellation_reasons_df.head()

,cr_id,reason,shortcut
0,90,Airline/ Carrier,A
1,80,Weather,B
2,70,National Air System,C
3,60,Security,D
4,0,Keine Stornierung,No


### **Anlegen und Befüllen der Tabelle `flights`**

Es werden aus der [Zeitreihe](https://www.kaggle.com/datasets/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018) die Daten für die Jahre 2014 und 2015 geladen, dieser Ausschnitt ist rein willkürlich. Hinweis: Das Attribut "tail_number" ist in den zugrunde liegenden Daten nicht vorhanden, es wird später aus einer anderen Datenquelle angereichert. Die "tail_numer" ist die individuelle Registrationsnummer eines jeden Flugzeugs.

**Load der Daten aus der Datei "2014.csv"**

In [7]:
# Anlegen der Tabelle "flights"
conn.execute("""
CREATE SEQUENCE IF NOT EXISTS "flight_id"
INCREMENT BY 1 MINVALUE 0;

CREATE TABLE IF NOT EXISTS flights 
(   "flight_id" INTEGER PRIMARY KEY DEFAULT(nextval('flight_id')),
    "date" DATE,
    "airline" VARCHAR(10),
    "flight_number" VARCHAR(255),
    "tail_number" VARCHAR(255),
    "origin_airport" VARCHAR(10),
    "destination_airport" VARCHAR(10),
    "scheduled_departure" VARCHAR(255),
    "departure_time" VARCHAR(255),
    "departure_delay" INTEGER,
    "taxi_out" INTEGER,
    "wheels_off" VARCHAR(255),
    "wheels_on" VARCHAR(255),
    "taxi_in" INTEGER,
    "scheduled_arrival" VARCHAR(255),    
    "arrival_time" VARCHAR(255),
    "arrival_delay" INTEGER,
    "cancelled" INTEGER,
    "cancellation_reason" VARCHAR(255),             
    "diverted" INTEGER,
    "scheduled_time" INTEGER,             
    "elapsed_time" INTEGER,             
    "air_time" INTEGER,
    "distance" INTEGER,
    "airline_delay" INTEGER,             
    "weather_delay" INTEGER,
    "air_system_delay" INTEGER,
    "security_delay" INTEGER,
    "late_aircraft_delay" INTEGER,
    CONSTRAINT "flight_id" UNIQUE ("flight_id")
);
""")

# Befüllen der Tabelle aus "2014.csv". Die Daten werden nicht über http aus Performance-Gründen, sondern über eine
# lokale csv-Datei geladen. Sie müssen den Pfad entsprechend dem Ihrigen anpassen
conn.execute("""
INSERT INTO flights
(
    "date",
    "airline",
    "flight_number",
    "origin_airport",
    "destination_airport",
    "scheduled_departure",
    "departure_time",
    "departure_delay",
    "taxi_out",
    "wheels_off",
    "wheels_on",
    "taxi_in",
    "scheduled_arrival",    
    "arrival_time",
    "arrival_delay",
    "cancelled",
    "cancellation_reason",             
    "diverted",
    "scheduled_time",             
    "elapsed_time",             
    "air_time",
    "distance",
    "airline_delay",             
    "weather_delay",
    "air_system_delay",
    "security_delay",
    "late_aircraft_delay"
)
SELECT 
"FL_DATE",
"OP_CARRIER",
"OP_CARRIER_FL_NUM",
"ORIGIN",
"DEST",
"CRS_DEP_TIME",
"DEP_TIME",
"DEP_DELAY",
"TAXI_OUT",
"WHEELS_OFF",
"WHEELS_ON",
"TAXI_IN",
"CRS_ARR_TIME",
"ARR_TIME",
"ARR_DELAY",
"CANCELLED",
"CANCELLATION_CODE",
"DIVERTED",
"CRS_ELAPSED_TIME",
"ACTUAL_ELAPSED_TIME",
"AIR_TIME",
"DISTANCE",
"CARRIER_DELAY",
"WEATHER_DELAY",
"NAS_DELAY",
"SECURITY_DELAY",
"LATE_AIRCRAFT_DELAY"
FROM read_csv_auto("C:/Users/rober/OneDrive/Vorlesungen/Datenbasierte Fallstudien/Visualisierungen/Flights/Flights/2014.csv");
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

**Load der Daten aus der Datei "2015.csv"**

In [8]:
# Anlegen der Tabelle "flights"
conn.execute("""
CREATE SEQUENCE IF NOT EXISTS "flight_id"
INCREMENT BY 1 MINVALUE 0;

CREATE TABLE IF NOT EXISTS flights 
(   "flight_id" INTEGER PRIMARY KEY DEFAULT(nextval('flight_id')),
    "date" DATE,
    "airline" VARCHAR(10),
    "flight_number" VARCHAR(255),
    "tail_number" VARCHAR(255),
    "origin_airport" VARCHAR(10),
    "destination_airport" VARCHAR(10),
    "scheduled_departure" VARCHAR(255),
    "departure_time" VARCHAR(255),
    "departure_delay" INTEGER,
    "taxi_out" INTEGER,
    "wheels_off" VARCHAR(255),
    "wheels_on" VARCHAR(255),
    "taxi_in" INTEGER,
    "scheduled_arrival" VARCHAR(255),    
    "arrival_time" VARCHAR(255),
    "arrival_delay" INTEGER,
    "cancelled" INTEGER,
    "cancellation_reason" VARCHAR(255),             
    "diverted" INTEGER,
    "scheduled_time" INTEGER,             
    "elapsed_time" INTEGER,             
    "air_time" INTEGER,
    "distance" INTEGER,
    "airline_delay" INTEGER,             
    "weather_delay" INTEGER,
    "air_system_delay" INTEGER,
    "security_delay" INTEGER,
    "late_aircraft_delay" INTEGER,
    CONSTRAINT "flight_id" UNIQUE ("flight_id")
);
""")

# Befüllen der Tabelle aus "2015.csv". Die Daten werden nicht über http aus Performance-Gründen, sondern über eine
# lokale csv-Datei geladen. Sie müssen den Pfad entsprechend dem Ihrigen anpassen
conn.execute("""
INSERT INTO flights
(
    "date",
    "airline",
    "flight_number",
    "origin_airport",
    "destination_airport",
    "scheduled_departure",
    "departure_time",
    "departure_delay",
    "taxi_out",
    "wheels_off",
    "wheels_on",
    "taxi_in",
    "scheduled_arrival",    
    "arrival_time",
    "arrival_delay",
    "cancelled",
    "cancellation_reason",             
    "diverted",
    "scheduled_time",             
    "elapsed_time",             
    "air_time",
    "distance",
    "airline_delay",             
    "weather_delay",
    "air_system_delay",
    "security_delay",
    "late_aircraft_delay"
)
SELECT 
"FL_DATE",
"OP_CARRIER",
"OP_CARRIER_FL_NUM",
"ORIGIN",
"DEST",
"CRS_DEP_TIME",
"DEP_TIME",
"DEP_DELAY",
"TAXI_OUT",
"WHEELS_OFF",
"WHEELS_ON",
"TAXI_IN",
"CRS_ARR_TIME",
"ARR_TIME",
"ARR_DELAY",
"CANCELLED",
"CANCELLATION_CODE",
"DIVERTED",
"CRS_ELAPSED_TIME",
"ACTUAL_ELAPSED_TIME",
"AIR_TIME",
"DISTANCE",
"CARRIER_DELAY",
"WEATHER_DELAY",
"NAS_DELAY",
"SECURITY_DELAY",
"LATE_AIRCRAFT_DELAY"
FROM read_csv_auto("C:/Users/rober/OneDrive/Vorlesungen/Datenbasierte Fallstudien/Visualisierungen/Flights/Flights/2015.csv");
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### **Anlegen und Befüllen der Tabelle `time`**

In [9]:
# Anlegen der Tabelle "time" als eigenständige Zeit-Dimension
conn.execute("""
    CREATE TABLE IF NOT EXISTS "time" (
    "date" DATE,
    "year" INTEGER,
    "month_int" INTEGER,
    "quarter" INTEGER,
    "week" INTEGER,
    "weekday" INTEGER,
    "weekday_name" VARCHAR(255),
    "quarter_name" VARCHAR(255),
    "month_name" VARCHAR(255),
    "month" VARCHAR(255),
    "week_name" VARCHAR(255)
);

-- Befüllen der Tabelle aus der Spalte date
INSERT INTO time ("date", "year", "month_int", "quarter", "week", "weekday", "weekday_name", "quarter_name", "month_name", "month", "week_name")
SELECT 
    DISTINCT "date",
    EXTRACT(YEAR FROM "date") AS "year",
    EXTRACT(MONTH FROM "date") AS "month_int",
    EXTRACT(QUARTER FROM "date") AS "quarter",
    EXTRACT(WEEK FROM "date") AS "week",
    EXTRACT(DOW FROM "date") AS "weekday",
    -- Berechne den abgekürzten Wochentagnamen
    CASE 
        WHEN EXTRACT(DOW FROM "date") = 0 THEN 'Sun'
        WHEN EXTRACT(DOW FROM "date") = 1 THEN 'Mon'
        WHEN EXTRACT(DOW FROM "date") = 2 THEN 'Tue'
        WHEN EXTRACT(DOW FROM "date") = 3 THEN 'Wed'
        WHEN EXTRACT(DOW FROM "date") = 4 THEN 'Thu'
        WHEN EXTRACT(DOW FROM "date") = 5 THEN 'Fri'
        WHEN EXTRACT(DOW FROM "date") = 6 THEN 'Sat'
    END AS weekday_name,
    -- Formatierung für Quartal
    'Q' || EXTRACT(QUARTER FROM "date") || ' ' || EXTRACT(YEAR FROM "date") AS "quarter_name",
    -- Formatierung für Monat
    CASE EXTRACT(MONTH FROM "date")
        WHEN 1 THEN 'Jan'
        WHEN 2 THEN 'Feb'
        WHEN 3 THEN 'Mar'
        WHEN 4 THEN 'Apr'
        WHEN 5 THEN 'May'
        WHEN 6 THEN 'Jun'
        WHEN 7 THEN 'Jul'
        WHEN 8 THEN 'Aug'
        WHEN 9 THEN 'Sep'
        WHEN 10 THEN 'Oct'
        WHEN 11 THEN 'Nov'
        WHEN 12 THEN 'Dec'
    END || ' ' || EXTRACT(YEAR FROM "date") AS "month_name",
    CASE EXTRACT(MONTH FROM "date")
        WHEN 1 THEN 'Jan'
        WHEN 2 THEN 'Feb'
        WHEN 3 THEN 'Mar'
        WHEN 4 THEN 'Apr'
        WHEN 5 THEN 'May'
        WHEN 6 THEN 'Jun'
        WHEN 7 THEN 'Jul'
        WHEN 8 THEN 'Aug'
        WHEN 9 THEN 'Sep'
        WHEN 10 THEN 'Oct'
        WHEN 11 THEN 'Nov'
        WHEN 12 THEN 'Dec'
    END AS month,
    -- Formatierung für Kalenderwoche
    'W' || EXTRACT(WEEK FROM "date") || ' ' || EXTRACT(YEAR FROM "date") AS "week_name"
FROM flights
ORDER BY "date";
""")


### **Data Preparation**

In [10]:
# Umformen der Werte wie "0054" in "00:54:00" und CAST in Datentype "TIME"
conn.execute("""
UPDATE flights
SET 
    "scheduled_departure" = CASE
        WHEN "scheduled_departure" IS NULL OR "scheduled_departure" = '' THEN NULL
        ELSE TRY_CAST(CONCAT(SUBSTRING("scheduled_departure", 1, 2), ':', SUBSTRING("scheduled_departure", 3, 2), ':00') AS TIME)
    END,
    "departure_time" = CASE
        WHEN "departure_time" IS NULL OR "departure_time" = '' THEN NULL
        ELSE TRY_CAST(CONCAT(SUBSTRING("departure_time", 1, 2), ':', SUBSTRING("departure_time", 3, 2), ':00') AS TIME)
    END,
    "wheels_off" = CASE
        WHEN "wheels_off" IS NULL OR "wheels_off" = '' THEN NULL
        ELSE TRY_CAST(CONCAT(SUBSTRING("wheels_off", 1, 2), ':', SUBSTRING("wheels_off", 3, 2), ':00') AS TIME)
    END,
    "wheels_on" = CASE
        WHEN "wheels_on" IS NULL OR "wheels_on" = '' THEN NULL
        ELSE TRY_CAST(CONCAT(SUBSTRING("wheels_on", 1, 2), ':', SUBSTRING("wheels_on", 3, 2), ':00') AS TIME)
    END,
    "scheduled_arrival" = CASE
        WHEN "scheduled_arrival" IS NULL OR "scheduled_arrival" = '' THEN NULL
        ELSE TRY_CAST(CONCAT(SUBSTRING("scheduled_arrival", 1, 2), ':', SUBSTRING("scheduled_arrival", 3, 2), ':00') AS TIME)
    END,
    "arrival_time" = CASE
        WHEN "arrival_time" IS NULL OR "arrival_time" = '' THEN NULL
        ELSE TRY_CAST(CONCAT(SUBSTRING("arrival_time", 1, 2), ':', SUBSTRING("arrival_time", 3, 2), ':00') AS TIME)
    END
WHERE "scheduled_departure" IS NOT NULL OR "departure_time" IS NOT NULL OR "wheels_off" IS NOT NULL OR "wheels_on" IS NOT NULL OR "scheduled_arrival" IS NOT NULL OR "arrival_time" IS NOT NULL;
""")

# Weitere Spalten in "flights" anlegen
conn.execute("""
ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "weekday_name" VARCHAR(255);

ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "quarter_name" VARCHAR(255);

ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "month_name" VARCHAR(255);

ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "week_name" VARCHAR(255);

ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "month_int" INTEGER;    

ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "month" VARCHAR(255);         

ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "year" INTEGER;
""")

# Aktualisieren der Spalten in "flights"
conn.execute("""
UPDATE flights
SET "weekday_name" = t."weekday_name"
FROM "time" t
WHERE t."date" = flights."date";

UPDATE flights
SET "week_name" = t."week_name"
FROM "time" t
WHERE t."date" = flights."date";

UPDATE flights
SET "month_name" = t."month_name"
FROM "time" t
WHERE t."date" = flights."date";

UPDATE flights
SET "month" = t."month"
FROM "time" t
WHERE t."date" = flights."date";  
             
UPDATE flights
SET "month_int" = t."month_int"
FROM "time" t
WHERE t."date" = flights."date";  

UPDATE flights
SET "quarter_name" = t."quarter_name"
FROM "time" t
WHERE t."date" = flights."date";
             
UPDATE flights
SET "year" = t."year"
FROM "time" t
WHERE t."date" = flights."date";             

UPDATE flights 
SET 
"cancellation_reason" = cr."reason"  
FROM cancellation_reasons cr
WHERE cr."shortcut" = flights."cancellation_reason";
""")

# Einfügen weiterer Spalten in "flights"
conn.execute("""
ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "origin_airport_lon" DOUBLE;

ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "origin_airport_lat" DOUBLE;

ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "destination_airport_lon" DOUBLE;

ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "destination_airport_lat" DOUBLE;

ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "origin_city" VARCHAR(255);

ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "destination_city" VARCHAR(255);

ALTER TABLE flights
ADD COLUMN IF NOT EXISTS "airline_name" VARCHAR(255);
""")

# Aktualisieren der Spalten
conn.execute("""
UPDATE flights 
SET "origin_airport_lon" = ap."longitude"
FROM airports ap
WHERE ap."iata_code" = flights."origin_airport";

UPDATE flights 
SET "origin_airport_lat" = ap."latitude"
FROM airports ap
WHERE ap."iata_code" = flights."origin_airport";

UPDATE flights 
SET "destination_airport_lon" = ap."longitude"
FROM airports ap
WHERE ap."iata_code" = flights."destination_airport";

UPDATE flights 
SET "destination_airport_lat" = ap."latitude"
FROM airports ap
WHERE ap."iata_code" = flights."destination_airport";

UPDATE flights 
SET "destination_city" = ap."city"
FROM airports ap
WHERE ap."iata_code" = flights."destination_airport";

UPDATE flights 
SET "origin_city" = ap."city"
FROM airports ap
WHERE ap."iata_code" = flights."origin_airport";

UPDATE flights 
SET "airline_name" = al."airline"
FROM airlines al
WHERE al."iata_code" = flights."airline";
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### **Erstellen von DB-Views und Überführen jeweils in ein `pandas DataFrame`**

**One Big Table**

Eine "One Big Table" ist ein Datenmodellierungskonzept, bei dem eine einzelne, denormalisierte Tabelle verwendet wird, um eine Vielzahl von Attributen oder Merkmalen zu speichern, die für analytische Abfragen oder Berichterstattungszwecke relevant sind. Dieses Modell aggregiert Daten aus mehreren Tabellen oder Datenquellen in einer einzigen Tabelle, indem es Normalisierungsprinzipien aufgibt, um eine vereinfachte Sicht zu schaffen, die schnelle und effiziente Abfragen ermöglicht.

In [11]:
conn.execute("""
CREATE VIEW IF NOT EXISTS vw_OneBigTable AS
SELECT
    "flight_id",
    "year",
    "date",
    "weekday_name",
    "quarter_name",
    "month_name",
    "month_int",
    "month",
    "week_name",    
    "airline" AS "iata_airline",
    "airline_name" AS "airline",
    "flight_number",
    "tail_number",
    "origin_airport",
    "origin_city",
    "origin_airport_lon",
    "origin_airport_lat",
    "destination_airport",
    "destination_city",
    "destination_airport_lon",
    "destination_airport_lat",
    "scheduled_departure",
    "departure_time",
    "departure_delay",
    "taxi_out",
    "wheels_off",
    "scheduled_time",
    "elapsed_time",
    "air_time",
    "distance"*1.60934 AS "distance_km",
    "wheels_on",
    "taxi_in",
    "scheduled_arrival",
    "arrival_time",
    "arrival_delay",
    "diverted",
    "cancelled",
    "cancellation_reason",
    "air_system_delay",
    "security_delay",
    "airline_delay",
    "late_aircraft_delay",
    "weather_delay"
FROM flights
ORDER BY "date" ASC;
""")

# Der auskommentierte Code würde die DuckDB-Sicht "vw_OneBigTable" in ein pandas DataFrame überführen, aber im Prinzip ist dies nicht nötig, da über DuckDB im Folgenden analytische Sichten vorbereitet wurden
# aus Performance-Sicht ist der Ansatz, analytische Sichten vorzubereiten ohnehin besser, als laufend eine One Big Table abzufragen

# Überführt die Daten aus der angelegten Relation in ein pandas DataFrame
#OBT_df = conn.execute("SELECT * FROM vw_OneBigTable;").df()

# Testabfrage
#OBT_df.head()

**Nur die durchgeführten ("cancelled = 0") Flüge als One Big Table. Diese dient als View für alle weiteren Analysen**

In [12]:
conn.execute("""
CREATE VIEW IF NOT EXISTS vw_ABT AS
SELECT * FROM vw_OneBigTable
WHERE "cancelled" = 0;
""")

**Analysen im Kontext "airlines"**

Annahme Verspätung: https://ansperformance.eu/definition/punctuality/#:~:text=The%20share%20of%20IFR%20flights,departure%20time%20(airline%20schedules).

Andere Quelle: *[A flight is counted as "on time" if it operated less than 15 minutes after the scheduled time shown in the carriers' Computerized Reservations Systems (CRS)](https://www.transportation.gov/sites/dot.gov/files/docs/2014DecemberATCR.pdf)*, p. 3.

In [13]:
conn.execute("""
CREATE VIEW IF NOT EXISTS vw_airlines_metrics AS
SELECT 
    "airline",
    "iata_airline",
    "year",
    "month",
    "month_int",            
    AVG("arrival_delay") AS "average_arrival_delay",
    MEDIAN("arrival_delay") AS "median_arrival_delay",
    AVG("departure_delay") AS "average_departure_delay",
    MEDIAN("departure_delay") AS "median_departure_delay",
    COUNT("flight_id") AS "total_flights",
    ROUND(SUM("distance_km"),0) AS distance_km,
    ROUND((SUM(CASE 
        WHEN ("departure_delay" >= 15) AND "cancelled" = 0 THEN 0
        ELSE 1
        END) / COUNT("flight_id")) * 100, 1) AS "percent of departures on time",
    ROUND((SUM(CASE
             WHEN ("arrival_delay" >=15 AND "cancelled" = 0) THEN 0
             ELSE 1
             END) / COUNT("flight_id")) * 100, 1) AS "percent of arrivals on time",
    ROUND((SUM(CASE 
        WHEN "cancelled" = 1 THEN 1
        ELSE 0
        END) / COUNT("flight_id")) * 100, 1) AS "cancellation_rate_percent",
    SUM("cancelled") AS "cancellations",
    (SELECT "destination_airport" 
        FROM "vw_OneBigTable" sub 
        WHERE sub."airline" = vw_OneBigTable."airline"
        GROUP BY "destination_airport"
        ORDER BY COUNT(*) DESC
        LIMIT 1) AS "most_frequent_destination",
    (SELECT "origin_airport" 
        FROM vw_OneBigTable sub 
        WHERE sub."airline" = vw_OneBigTable."airline"
        GROUP BY "origin_airport"
        ORDER BY COUNT(*) DESC
        LIMIT 1) AS "most_frequent_origin",
     ROUND((AVG("elapsed_time")- AVG("scheduled_time")),1) AS "buffer_flight"
FROM vw_OneBigTable
GROUP BY "airline", "iata_airline", "year", "month", "month_int"
ORDER BY "average_arrival_delay";
""")

# Überführt die Daten aus der angelegten Relation in ein pandas DataFrame
airlines_metrics_df = conn.execute("SELECT * FROM vw_airlines_metrics;").df()

# Testabfrage
airlines_metrics_df.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,airline,iata_airline,year,month,month_int,average_arrival_delay,median_arrival_delay,average_departure_delay,median_departure_delay,total_flights,distance_km,percent of departures on time,percent of arrivals on time,cancellation_rate_percent,cancellations,most_frequent_destination,most_frequent_origin,buffer_flight
0,None,FL,2014,Nov,11,-5.172423,-9.0,2.642803,-2.0,3951,4074715.0,88.9,90.7,0.3,12.0,None,None,-7.9
1,Delta Air Lines Inc.,DL,2015,Oct,10,-5.011516,-9.0,3.215743,-2.0,75552,101764164.0,91.3,92.2,0.0,17.0,ATL,ATL,-8.2
2,Alaska Airlines Inc.,AS,2014,Jan,1,-3.973786,-8.0,0.724945,-4.0,12403,23756812.0,91.4,89.7,1.2,152.0,SEA,SEA,-3.1
3,American Eagle Airlines Inc.,MQ,2015,Oct,10,-3.735054,-11.0,3.457587,-4.0,21982,14152764.0,89.4,89.6,1.2,273.0,ORD,ORD,-7.1
4,Delta Air Lines Inc.,DL,2015,Sep,9,-3.703377,-9.0,3.514294,-2.0,72063,96742254.0,90.6,90.7,0.0,6.0,ATL,ATL,-7.2


**Analysen im Kontext "flights"**

In [14]:
conn.execute("""
CREATE VIEW IF NOT EXISTS vw_flight_metrics AS
SELECT 
   "tail_number",
   "airline",
   "origin_airport",
   "year",
   "month",
   COUNT("flight_id") AS "total_flights",
   ROUND(SUM("distance_km"),0) AS "total_km",
   ROUND((AVG("distance_km") / AVG("air_time" / 60.0)),1) AS "average_speed_km"
FROM vw_ABT
GROUP BY "origin_airport", "tail_number", "airline", "year", "month"
ORDER BY "total_flights" DESC, "total_km" DESC;
""")

# Überführt die Daten aus der angelegten Relation in ein pandas DataFrame
flight_metrics_df = conn.execute("SELECT * FROM vw_flight_metrics;").df()

# Testabfrage
flight_metrics_df.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,tail_number,airline,origin_airport,year,month,total_flights,total_km,average_speed_km
0,None,Delta Air Lines Inc.,ATL,2015,Aug,21624,23935093.0,701.5
1,None,Delta Air Lines Inc.,ATL,2015,Jul,21579,23941435.0,698.5
2,None,Delta Air Lines Inc.,ATL,2015,Oct,20968,22456069.0,688.9
3,None,Delta Air Lines Inc.,ATL,2015,Mar,20897,22451262.0,677.0
4,None,Delta Air Lines Inc.,ATL,2015,May,20835,22354565.0,686.7


**Analysen im Kontext "flight routes"**

In [15]:
conn.execute("""
CREATE VIEW IF NOT EXISTS vw_flight_routes AS
SELECT
            "origin_airport",
            "destination_airport",
            "origin_city",
            "destination_city",
            "origin_airport_lon",
            "origin_airport_lat",
            "destination_airport_lon",
            "destination_airport_lat",
            "year",
            "month",
            COUNT("flight_id"),
            ROUND(SUM("distance_km"),0),
            ROUND(AVG("distance_km"),0) AS avg_distance_km
FROM vw_ABT
GROUP BY
            "origin_airport",
            "destination_airport",
            "origin_city",
            "destination_city",
            "origin_airport_lon",
            "origin_airport_lat",
            "destination_airport_lon",
            "destination_airport_lat",
            "year",
            "month";
""")

flight_routes_df = conn.execute("""SELECT * FROM vw_flight_routes;""").df()

# Testabfrage
flight_routes_df.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,origin_airport,destination_airport,origin_city,destination_city,origin_airport_lon,origin_airport_lat,destination_airport_lon,destination_airport_lat,year,month,count(flight_id),"round(sum(distance_km), 0)",avg_distance_km
0,DFW,JAC,Dallas-Fort Worth,Jackson,-97.03720,32.89595,-110.73774,43.60732,2015,Dec,29,48864.0,1685.0
1,ORD,RDU,Chicago,Raleigh,-87.90446,41.97960,-78.78747,35.87764,2015,Dec,169,175698.0,1040.0
2,DFW,DTW,Dallas-Fort Worth,Detroit,-97.03720,32.89595,-83.34884,42.21206,2015,Dec,213,337990.0,1587.0
3,DFW,LAS,Dallas-Fort Worth,Las Vegas,-97.03720,32.89595,-115.15233,36.08036,2015,Dec,359,609529.0,1698.0
4,SFO,DFW,San Francisco,Dallas-Fort Worth,-122.37484,37.61900,-97.03720,32.89595,2015,Dec,421,991907.0,2356.0


**Analysen im Kontext "cancellations"**

In [16]:
conn.execute("""
CREATE VIEW IF NOT EXISTS vw_cancellations AS
SELECT
    "airline",
    "cancelled",
    "cancellation_reason",
    "year",
    "month",
    CAST(SUM("cancelled") AS INT) AS "cancellations",
    ROUND(SUM("cancelled")::FLOAT/ COUNT(*) *100, 1) AS "cancellation_rate"
FROM vw_OneBigTable
GROUP BY
            "airline",
            "cancelled",
            "cancellation_reason",
            "year",
            "month";
""")

# Überführt die Daten aus der angelegten Relation in ein pandas DataFrame
cancellations_df = conn.execute("SELECT * FROM vw_cancellations;").df()

cancellations_df.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,airline,cancelled,cancellation_reason,year,month,cancellations,cancellation_rate
0,Delta Air Lines Inc.,0,None,2015,Dec,0,0.0
1,United Air Lines Inc.,1,Weather,2015,Dec,565,100.0
2,United Air Lines Inc.,1,Airline/ Carrier,2015,Dec,211,100.0
3,Atlantic Southeast Airlines,1,Airline/ Carrier,2015,Dec,266,100.0
4,Delta Air Lines Inc.,0,None,2015,May,0,0.0


**Analysen im Kontext "time"**

In [17]:
conn.execute("""
CREATE VIEW IF NOT EXISTS vw_time_analysis AS
SELECT 
    "year", 
    "quarter_name", 
    "month_name", 
    "weekday_name", 
    "airline",
    "month_int", 
    "month",
    COUNT(*) AS total_flights,
    ROUND(MEDIAN("departure_delay"), 1) AS med_departure_delay,
    ROUND(MEDIAN("arrival_delay"), 1) AS med_arrival_delay,
    "cancelled", 
    "cancellation_reason"
FROM vw_OneBigTable
GROUP BY 
    "year", 
    "quarter_name", 
    "month_name", 
    "weekday_name", 
    "airline",
    "month_int",
    "month",
    "cancelled", 
    "cancellation_reason"
HAVING AVG("departure_delay") > 15;
""")

# Überführt die Daten aus der angelegten Relation in ein pandas DataFrame
time_df = conn.execute("""SELECT * FROM vw_time_analysis""").df()

# Testabfrage
time_df.head(15)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,year,quarter_name,month_name,weekday_name,airline,month_int,month,total_flights,med_departure_delay,med_arrival_delay,cancelled,cancellation_reason
0,2015,Q4 2015,Dec 2015,Wed,Southwest Airlines Co.,12,Dec,46,-1.0,NaN,1,Weather
1,2015,Q4 2015,Dec 2015,Thu,Atlantic Southeast Airlines,12,Dec,115,289.0,NaN,1,Weather
2,2015,Q4 2015,Dec 2015,Thu,Skywest Airlines Inc.,12,Dec,20,79.0,NaN,1,Airline/ Carrier
3,2015,Q4 2015,Dec 2015,Fri,Virgin America,12,Dec,725,-1.0,-3.0,0,None
4,2015,Q4 2015,Dec 2015,Sun,United Air Lines Inc.,12,Dec,5520,1.0,-7.0,0,None
5,2015,Q4 2015,Dec 2015,Mon,Southwest Airlines Co.,12,Dec,13848,1.0,-3.0,0,None
6,2015,Q4 2015,Dec 2015,Tue,Skywest Airlines Inc.,12,Dec,21,403.0,NaN,1,Airline/ Carrier
7,2015,Q4 2015,Dec 2015,Wed,United Air Lines Inc.,12,Dec,57,2.0,NaN,1,Airline/ Carrier
8,2015,Q4 2015,Dec 2015,Thu,Spirit Air Lines,12,Dec,9,18.0,NaN,1,National Air System
9,2015,Q4 2015,Dec 2015,Sat,Southwest Airlines Co.,12,Dec,174,108.0,NaN,1,Weather


### **Beenden der Verbindung zur Datenbank**

In [18]:
# Beendet die Verbindung zur im Arbeitsspeicher liegenden in-Memory-DB. Dabei gehen auch alle Daten im Arbeitsspeicher verloren. Die DataFrames bleiben für die Dauer der Arbeitssitzung erhalten
conn.close()

## **Datenvisualisierung und -analyse**

Wir haben die Verbindung zur in-Memory DB DuckDB geschlossen (spart Ressourcen) und führen nun die Analysen mit `pandas` durch. Über DuckDB haben wir DB-Views für die Analysen vorbereitet und jeweils in ein DataFrame überführt. Diese dienen als Grundlage für die weiterführenden Analysen.  

In [19]:
# Auflisten aller DataFrames
dfs = [var for var in globals() if isinstance(eval(var), pd.core.frame.DataFrame)]
print(dfs)

['_', '__', '___', 'airports_df', '_4', 'airlines_df', '_5', 'cancellation_reasons_df', '_6', 'airlines_metrics_df', '_13', 'flight_metrics_df', '_14', 'flight_routes_df', '_15', 'cancellations_df', '_16', 'time_df', '_17']


### **Statistiken zu den Fluggesellschaften**

**Anzahl der Stornierungen pro Fluggesellschaft**

In [20]:
cancellations_summary = cancellations_df.groupby(['cancellation_reason', 'airline', 'year', 'month'])['cancellations'].sum().reset_index()
cancellations_summary['percentage'] = (cancellations_summary['cancellations'] / cancellations_summary['cancellations'].sum()) * 100

# DataFrame als CSV speichern
cancellations_summary.to_csv(r"C:\Users\rober\OneDrive\Vorlesungen\Datenbasierte Fallstudien\Visualisierungen\Flights\Flights\cancellations_summary.csv", index=False)

cancellations_summary.head()

,cancellation_reason,airline,year,month,cancellations,percentage
0,Airline/ Carrier,Alaska Airlines Inc.,2014,Apr,17,0.007894
1,Airline/ Carrier,Alaska Airlines Inc.,2014,Aug,29,0.013466
2,Airline/ Carrier,Alaska Airlines Inc.,2014,Dec,30,0.013931
3,Airline/ Carrier,Alaska Airlines Inc.,2014,Feb,56,0.026004
4,Airline/ Carrier,Alaska Airlines Inc.,2014,Jan,26,0.012073


In [21]:
flight_routes_df.to_csv(r"C:\Users\rober\OneDrive\Vorlesungen\Datenbasierte Fallstudien\Visualisierungen\Flights\Flights\flight_routes_summary.csv", index=False)

In [22]:
flight_routes_df

,origin_airport,destination_airport,origin_city,destination_city,origin_airport_lon,origin_airport_lat,destination_airport_lon,destination_airport_lat,year,month,count(flight_id),"round(sum(distance_km), 0)",avg_distance_km
0,DFW,JAC,Dallas-Fort Worth,Jackson,-97.03720,32.89595,-110.73774,43.60732,2015,Dec,29,48864.0,1685.0
1,ORD,RDU,Chicago,Raleigh,-87.90446,41.97960,-78.78747,35.87764,2015,Dec,169,175698.0,1040.0
2,DFW,DTW,Dallas-Fort Worth,Detroit,-97.03720,32.89595,-83.34884,42.21206,2015,Dec,213,337990.0,1587.0
3,DFW,LAS,Dallas-Fort Worth,Las Vegas,-97.03720,32.89595,-115.15233,36.08036,2015,Dec,359,609529.0,1698.0
4,SFO,DFW,San Francisco,Dallas-Fort Worth,-122.37484,37.61900,-97.03720,32.89595,2015,Dec,421,991907.0,2356.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97529,MDW,PNS,Chicago,Pensacola,-87.75242,41.78598,-87.18744,30.47331,2015,Jul,4,5028.0,1257.0
97530,RAP,IAH,Rapid City,Houston,-103.05737,44.04532,-95.33972,29.98047,2015,Jul,8,13621.0,1703.0
97531,MIA,TUL,Miami,Tulsa,-80.29056,25.79325,-95.88824,36.19837,2015,Jul,4,7519.0,1880.0
97532,BGR,LGA,Bangor,New York,-68.82814,44.80744,-73.87261,40.77724,2015,Jul,1,608.0,608.0


In [23]:
airlines_summary = airlines_metrics_df.groupby(['airline', 'month', 'month_int', 'year'])[['total_flights', 'percent of arrivals on time', 'percent of departures on time', 'cancellation_rate_percent']].sum().reset_index()

# DataFrame als CSV speichern
airlines_summary.to_csv(r"C:\Users\rober\OneDrive\Vorlesungen\Datenbasierte Fallstudien\Visualisierungen\Flights\Flights\airlines_summary.csv", index=False)

airlines_summary.head()

,airline,month,month_int,year,total_flights,percent of arrivals on time,percent of departures on time,cancellation_rate_percent
0,Alaska Airlines Inc.,Apr,4,2014,12672,90.7,92.9,0.2
1,Alaska Airlines Inc.,Apr,4,2015,13974,89.9,91.4,0.5
2,Alaska Airlines Inc.,Aug,8,2014,15134,84.5,87.8,0.4
3,Alaska Airlines Inc.,Aug,8,2015,16095,83.3,86.2,0.2
4,Alaska Airlines Inc.,Dec,12,2014,13611,81.3,84.9,0.5


## **Erste Auswertungen mit interaktivem Analytics-Tool**

In [ ]:
%%capture
%pip install pygwalker

In [26]:
import pygwalker as pyg

walker = pyg.walk(cancellations_df)

Box(children=(HTML(value='<div id="ifr-pyg-000616f25b45cbd2vF4sJQtSwTXI0yL8" style="height: auto">\n    <head>…

## **Erstellen einer dash-Anwendung**

Da es einigen konfigurativen Aufwand bedeutet, eine dash-App aus einem Jupyter-Notebook heraus zu starten, wird eine eigene py-Datei dafür aufgesetzt, die sich leichter als dash-App deployen lässt. 